Import header and cookie information to sign into goodreads

In [17]:
# Asynchronous
import nest_asyncio
nest_asyncio.apply()
import asyncio
import aiohttp
import json

import tqdm.asyncio
import requests
from pprint import pprint
import datetime
from bs4 import BeautifulSoup

import pandas as pd
import numpy as np
import json

import re
cookies = {
    'ccsid': '027-9406763-1350835',
    '__qca': 'P0-867993267-1671835417644',
    'p': 'BFnM5Q76wF0shLrczPEWD6Ai6glvalBOI0HQnM7wAheYffUa',
    'likely_has_account': 'true',
    'srb_8': '0_ar',
    'locale': 'en',
    'csm-sid': '438-4025740-7454350',
    'allow_behavioral_targeting': 'true',
    'session-id': '147-4067731-5921029',
    'lc-main': 'en_US',
    'logged_out_browsing_page_count': '2',
    'ubid-main': '131-6414756-0108314',
    'csm-hit': 'tb:9FKSJEW3PZPAQZKSV74Z+s-9FKSJEW3PZPAQZKSV74Z|1675549894918&t:1675549894919&adb:adblk_yes',
    'session-id-time': '2306269930l',
    'session-token': 'fmlMtndkzGiWYItedsruMqFs9+7dbrKkjWxwS050IWs57BCY8RQJGXxU8qPn+6lNucog+VtoV/qrLn6eBpe8GwFqeSA75LzytrwlvgKScLWr62XYbJJg36UcoVnTCNaBAUkT+jhZdWNLpHXEQ/T3fsq03ctsMB7GChLcVrF/10rEk0ETQ5LmuUN5ordUvBjgbgf/wLazQPbG5Ia8rmhh/Gyptwz9N0ilyQSLx5h7pVc1fl4z+U3O3seFJdO2xI8M',
    'x-main': '"QKUGsTWzgOIn1MQ?MLW6M7?W9sXe6vT8lOfYIRdLo30O9fDyRRuxqM6Up6QliSso"',
    'at-main': 'Atza|IwEBIE9-j0Q56LUPug78YuRWyKfqUI-71kqBUKBSv9o0pgXoEdWRTp3QanY2PVmzS5kuRKhJ-2Qf2Y2Xza8aOzNTehTteyzG_d1HL64CDYGBvcFiM3J3K6q-_iEKTHPqIuFGTC7GBQffrkFancRYWCUTNK7ii9qpL1Qvc-lEAmj5xJFsos2I98msXxsTvgVFMUfr1rhJK6CfvkpCAIziz2lPcltlQCWaFsR10_Le_ytyKVYeIZ18m-F1bXhhejn7mTwQJl0',
    'sess-at-main': '"f/c/JR/xicYEmsJ7sGnJySc4uAQ6ojXuaOFFChXMGho="',
    '_session_id2': '64c06a0ae25fee31093af8a3bf3f79e1',
}

headers = {
    'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,image/avif,image/webp,image/apng,/;q=0.8,application/signed-exchange;v=b3;q=0.9',
    'Accept-Language': 'en-US,en;q=0.9',
    'Cache-Control': 'max-age=0',
    'Connection': 'keep-alive',
    # 'Cookie': 'ccsid=027-9406763-1350835; __qca=P0-867993267-1671835417644; p=BFnM5Q76wF0shLrczPEWD6Ai6glvalBOI0HQnM7wAheYffUa; likely_has_account=true; srb_8=0_ar; locale=en; csm-sid=438-4025740-7454350; allow_behavioral_targeting=true; session-id=147-4067731-5921029; lc-main=en_US; logged_out_browsing_page_count=2; ubid-main=131-6414756-0108314; csm-hit=tb:9FKSJEW3PZPAQZKSV74Z+s-9FKSJEW3PZPAQZKSV74Z|1675549894918&t:1675549894919&adb:adblk_yes; session-id-time=2306269930l; session-token=fmlMtndkzGiWYItedsruMqFs9+7dbrKkjWxwS050IWs57BCY8RQJGXxU8qPn+6lNucog+VtoV/qrLn6eBpe8GwFqeSA75LzytrwlvgKScLWr62XYbJJg36UcoVnTCNaBAUkT+jhZdWNLpHXEQ/T3fsq03ctsMB7GChLcVrF/10rEk0ETQ5LmuUN5ordUvBjgbgf/wLazQPbG5Ia8rmhh/Gyptwz9N0ilyQSLx5h7pVc1fl4z+U3O3seFJdO2xI8M; x-main="QKUGsTWzgOIn1MQ?MLW6M7?W9sXe6vT8lOfYIRdLo30O9fDyRRuxqM6Up6QliSso"; at-main=Atza|IwEBIE9-j0Q56LUPug78YuRWyKfqUI-71kqBUKBSv9o0pgXoEdWRTp3QanY2PVmzS5kuRKhJ-2Qf2Y2Xza8aOzNTehTteyzG_d1HL64CDYGBvcFiM3J3K6q-_iEKTHPqIuFGTC7GBQffrkFancRYWCUTNK7ii9qpL1Qvc-lEAmj5xJFsos2I98msXxsTvgVFMUfr1rhJK6CfvkpCAIziz2lPcltlQCWaFsR10_Le_ytyKVYeIZ18m-F1bXhhejn7mTwQJl0; sess-at-main="f/c/JR/xicYEmsJ7sGnJySc4uAQ6ojXuaOFFChXMGho="; _session_id2=64c06a0ae25fee31093af8a3bf3f79e1',
    'If-None-Match': 'W/"087d62f3d9fcbed8836cb66e1e0c2bef"',
    'Referer': 'https://www.goodreads.com/',
    'Sec-Fetch-Dest': 'document',
    'Sec-Fetch-Mode': 'navigate',
    'Sec-Fetch-Site': 'same-origin',
    'Sec-Fetch-User': '?1',
    'Upgrade-Insecure-Requests': '1',
    'User-Agent': 'Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/109.0.0.0 Safari/537.36',
    'sec-ch-ua': '"Not_A Brand";v="99", "Google Chrome";v="109", "Chromium";v="109"',
    'sec-ch-ua-mobile': '?0',
    'sec-ch-ua-platform': '"macOS"',
}

params = {
    'ref': 'nav_comm_people',
}

Open file with all userids

In [3]:
f = open('users.json')
users = json.load(f)

Get user details

In [ ]:
async def get_user_review_pages(session, user):
    async with session.get("https://www.goodreads.com//user/show/"+user,
                                 params=params, cookies=cookies, headers=headers) as response:
        ur_page = await response.text()
        ur_soup = BeautifulSoup(ur_page, 'html.parser')
        if ur_soup.find("div", class_="infoBoxRowItem") == None:
            return(user)
        
        details = ur_soup.find("div", class_="infoBoxRowItem").text
        
        details_user = []
        details_user.append((details,user))
        all_user_details.extend(details_user)
        

        # Finalize
        print("READ", end=' ')
        return("")

async def download_comments():
    async with aiohttp.ClientSession(timeout = aiohttp.ClientTimeout(total=60*60*24)) as session:

        tasks = []
        for user in users:
            tasks.append(asyncio.ensure_future(get_user_review_pages(session, user)))

        failed_users = await asyncio.gather(*tasks)
        failed_users_res = []
        for f_user in failed_users:
            if f_user != "":
                failed_users_res.append(f_user)

        return(failed_users_res)

all_user_details = []
failed_users = asyncio.run(download_comments())

with open('user_details.json', 'w') as file:
    json.dump(all_user_details, file, indent=2)

Getting countries with the userids

In [14]:
user_types = ['https://www.goodreads.com/user/best_reviewers', 'https://www.goodreads.com/user/top_reviewers','https://www.goodreads.com/user/top_readers']
countries = ['?country=CA','?country=all','?country=US','?country=US','?country=AU','?country=SC', '?country=NZ','?country=IE','?country=GB', '?country=SG', '?country=GB','?country=DO', '?country=TT', '?country=MT', '?country=BB','?country=LC','?country=GY']
time_frame = ['&duration=w','&duration=m', '&duration=y','&duration=a']

users = set()
for type in user_types: 
    for country in countries:
        for time in time_frame:
            url = type+country+time
            response = requests.get(url, params=params, cookies=cookies, headers=headers)
            soup = BeautifulSoup(response.content, 'html.parser')
            tab = soup.find("table",{"class":"tableList"})
            if tab is None:
                continue
            else:
                trs = tab.find_all('tr')
                for t in trs: 
                    s = t.find_all('td')[1].find('a',href = True)['href']
                    user_id = re.findall(r'\d+',s)[0]
                    user_loc = [user_id, country]
                    users.add(tuple(user_loc))

In [11]:
g = open('user_details.json')
user_details = json.load(g)

In [19]:
with open('user_countries.json', 'w') as file:
    json.dump(list(users), file, indent=2)

In [ ]:
user_details_new = []
for user in user_details:
    age = ""
    gender = ""
    user_id = user[-1:][0]
    print(user_id)
    for item in user: 
        items = item.split(',')
        for i in items: 
           
            if 'Age ' in i: 
                age = re.findall(r'\d+',i)[0]
            if 'Female' in i or 'Male' in i: 
                gender = i.strip()
                
    user_deets = [user_id, age, gender]
    user_details_new.append(user_deets)
   

In [48]:
import csv
headers = ['USERID', 'age', 'gender']
with open('user_details.csv', 'w') as file:
    w = csv.writer(file)
    w.writerow(headers)
    w.writerows(user_details_new)

In [57]:
user_details_new = pd.read_csv('user_details.csv')
user_details_new['USERID'] = user_details_new['USERID'].astype(int)
user_countries = pd.DataFrame(json.load(open('user_countries.json')))
user_countries.columns = ['USERID', 'Country']
user_countries['USERID'] = user_countries['USERID'].astype(int)

In [66]:
user_details_final = user_details_new.merge(user_countries,'left')
user_details_final['Country'] = user_details_final['Country'].str[-2:]

In [69]:
user_details_final.to_csv('user_details.csv', index=False)

Users and Books in one file

In [71]:
bookshelves = json.load(open('users_bookshelves.json'))

In [78]:
bookshelve_new = []
for book in bookshelves:
    book_id = book[0].split('/')[3]
    book_id = re.findall(r'\d+',book_id)[0]
    user_id = book[1]
    bookshelve_new.append([book_id,user_id])
    

In [80]:
headers = ['BOOKID', 'USERID']
with open('bookshelves.csv', 'w') as file:
    w = csv.writer(file)
    w.writerow(headers)
    w.writerows(bookshelve_new)